In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from funcions import quadrant_angle, quadrant_vector, canvi_quadrant, find_last_string_with_text, dist_lim_normalitat

In [2]:
if False:
    reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi", chunksize=1)
    df_dades = reader.get_chunk()
    print(df_dades)
    print(df_dades.columns)

    count=0
    for i in df_dades.columns:
        print(i, count)
        count+=1


#reader = pd.read_csv("../../dades/dades posturograf/2022071305_sve_FJ52F.txt", delimiter="\t", encoding="ansi", chunksize=2)
reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None)

df_dades = reader.get_chunk()
#print(df_dades)
#print(df_dades.columns)
#num_test = df_dades[['Nº_ROA','Nº_ROC','Nº_RGA','Nº_RGC']].values.tolist()

#'Nº_test', 'Val_test',Rep_test', 'EstML_test', 'EstAP_test'
roa = df_dades.iloc[:,16:21].values.tolist()
roc = df_dades.iloc[:,21:26].values.tolist()
rga = df_dades.iloc[:,26:31].values.tolist()
rgc = df_dades.iloc[:,31:36].values.tolist()

##SOM VIS VEST
result = df_dades.iloc[:,53:56].values.tolist()

print(roa)
print(roc)
print(rga)
print(rgc)
print(result)

num_test = roa[0][0] + roc[0][0] + rga[0][0] + rgc[0][0]
print("Total tests")
print(num_test)

LookupError: unknown encoding: ansi

In [ ]:
reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="cp1252", skiprows=2, chunksize=num_test, index_col=False)
df_resultats = reader.get_chunk()

df_resultats


In [ ]:
df_resultats['test'] = df_resultats['Prueba ROMBERG'] + df_resultats['Numero'].astype(str)
df_resultats

In [ ]:
reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi",skiprows=(17+int(num_test)), chunksize=1200, index_col=False)
df_dades_postu_XY = reader.get_chunk()
df_dades_postu_XY

In [ ]:
df_dades_postu_FxFy  = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_postu_IMU/2023031504_sve_VM54F.txt", delimiter="\t", encoding="ansi",skiprows=(17+int(num_test)+1+1200), index_col=False)
df_dades_postu_FxFy

# Càlcul paràmetres posturògraf

In [ ]:
params = {'Desplaç_total' : [], 'Angle_total' : [], 'Dispersio_ML' : [], 'Dispersio_AP' : [], 'Area_barrida' : []
         , 'V_mitjana' : [], 'Desplaç_ML' : [], 'Desplaç_AP' : [], 'Fmax_ML' : [], 'Fmax_AP' : []} 

## Càlcul desplaçament total

Càlcul del centre del núvol de punts(mitjana).

In [ ]:
llista_test = df_resultats['test'].tolist()
punts_X = {}
punts_Y = {}

for item in llista_test:
    punts_X[item] = df_dades_postu_XY[item + "_X"].values.tolist()
    punts_Y[item] = df_dades_postu_XY[item+"_Y"].values.tolist()
    params['Desplaç_total'].append(np.sqrt((sum(punts_X[item])/len(punts_X[item]))**2 + (sum(punts_Y[item])/len(punts_Y[item]))**2))

## Càlcul Angle total

Calcular angle del vector del centre del núvol de punts, cambiar quadrant al que pertany.

In [ ]:
for item in llista_test:
    vector_nuvp = [(sum(punts_X[item])/len(punts_X[item])),(sum(punts_Y[item])/len(punts_Y[item]))]
    
    quadrant_to = quadrant_vector(vector_nuvp)
    angle = math.degrees(math.atan(vector_nuvp[1]/vector_nuvp[0]))
    quadrant_from = quadrant_angle(angle)
    
    params['Angle_total'].append(canvi_quadrant(angle, quadrant_from, quadrant_to))

## Càlcul dispersió ML i AP

Calcular la desviació estàndard dels punts en X i en Y per separat.

In [ ]:
for item in llista_test:
    centre_X = sum(punts_X[item])/(len(punts_X[item])-1)
    centre_Y = sum(punts_Y[item])/(len(punts_Y[item])-1)
    params['Dispersio_ML'].append(np.sqrt(sum([((x - centre_X) ** 2) for x in punts_X[item]]) / (len(punts_X[item])-1)))
    params['Dispersio_AP'].append(np.sqrt(sum([((y - centre_Y) ** 2) for y in punts_Y[item]]) / (len(punts_Y[item])-1)))

## Càlcul Àrea barrida

Trobar valors propis de matriu de covariances de les dades.

In [ ]:
for item in llista_test:
    points = np.vstack((punts_X[item], punts_Y[item])).T
    dades_centrades = points - np.mean(points, axis=0)
    covariance_matrix = np.cov(dades_centrades, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    params['Area_barrida'].append(np.sqrt(eigenvalues[0]*eigenvalues[1])*4)

## Càlcul velocitat mitjana

Calcular la distància total recorreguda sumant la distància entre tots els punts consecutius, passar-ho a metres i dividir entre el temps de la prova.

In [ ]:
for item in llista_test:
    points = np.vstack((punts_X[item], punts_Y[item])).T
    num_points = len(points)
    total_dist = 0.0

    for i in range(num_points - 1):
        total_dist += np.linalg.norm(points[i+1] - points[i])
            
    params['V_mitjana'].append((total_dist/1000)/30)

## Càlcul desplaçament ML i AP

Distància entre el major i el menor punt en cada eix.

In [ ]:
for item in llista_test:
    params['Desplaç_ML'].append(np.max(punts_X[item]) - np.min(punts_X[item]))
    params['Desplaç_AP'].append(np.max(punts_Y[item]) - np.min(punts_Y[item]))

## Càlcul força màxima

Diferència entre el punt de més i de menys força en cada eix.

In [ ]:
for item in llista_test:
    forca_X = df_dades_postu_FxFy[item + "_Fx"].values.tolist()
    forca_Y = df_dades_postu_FxFy[item+"_Fy"].values.tolist()
    params['Fmax_ML'].append(max(list(map(float,forca_X[0:1200]))) - min(list(map(float,forca_X[0:1200]))))
    params['Fmax_AP'].append(max(list(map(float,forca_X[0:1200]))) - min(list(map(float,forca_X[0:1200]))))

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
# Calcular l'R²
r2 = []

r2.append(r2_score(df_resultats['Desplaz.Total(mm)'], params['Desplaç_total']))
r2.append(r2_score(df_resultats['Angulo Desplaz.(º)'], params['Angle_total']))
r2.append(r2_score(df_resultats['Dispers ML (mm)'], params['Dispersio_ML']))
r2.append(r2_score(df_resultats['Dispers AP (mm)'], params['Dispersio_AP']))
r2.append(r2_score(df_resultats['Area barrida (mm2)'], params['Area_barrida']))
r2.append(r2_score(df_resultats['Velocidad media (m/s)'], params['V_mitjana']))
r2.append(r2_score(df_resultats['Desplazam.ML (mm)'], params['Desplaç_ML']))
r2.append(r2_score(df_resultats['Desplazam.AP(mm)'], params['Desplaç_AP']))
r2.append(r2_score(df_resultats['Fuerza Max ML (N)'], params['Fmax_ML']))
r2.append(r2_score(df_resultats['Fuerza Max AP (N)'], params['Fmax_AP']))

## Valoració final

In [ ]:
def find_last_string_with_text(text, llista):
    for string in llista[::-1]:
        if text in string:
            return string
    return None

In [ ]:
import os

norm_ROAx = []
norm_ROAy = []

norm_ROCx = []
norm_ROCy = []

norm_RGAx = []
norm_RGAy = []

norm_RGCx = []
norm_RGCy = []

not_norm_ROAx = []
not_norm_ROAy = []

not_norm_ROCx = []
not_norm_ROCy = []

not_norm_RGAx = []
not_norm_RGAy = []

not_norm_RGCx = []
not_norm_RGCy = []

edats = []

for filename in os.listdir("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf"):
    if filename.endswith('.txt'):
        reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf/" + filename, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None)
        df_dades = reader.get_chunk()
        if df_dades[[9]].values[0][0] > 67:
            edats.append(df_dades[[9]].values[0][0])
            num_test = df_dades[[16]].values[0] + df_dades[[21]].values[0] + df_dades[[26]].values[0] + df_dades[[31]].values[0]
            df_dades_postu_FxFy = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf/" + filename, delimiter="\t", encoding="ansi",skiprows=(17+int(num_test)+1+1200), index_col=False)
            if isinstance(df_dades[[17]].values[0][0], float) and df_dades[[17]].values[0][0] > 90:
                valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROA", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROA", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                norm_ROAy.append(max(valorsy)-min(valorsy))
                norm_ROAx.append(max(valorsx)-min(valorsx))
            else:
                if 'ROA' in '\t'.join(df_dades_postu_FxFy.columns.tolist()[:-1]):                
                    valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROA", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROA", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                    not_norm_ROAy.append(max(valorsy)-min(valorsy))
                    not_norm_ROAx.append(max(valorsx)-min(valorsx))
                    
            if isinstance(df_dades[[22]].values[0][0], float) and df_dades[[22]].values[0][0] > 90:
                valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROC", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))                            
                valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROC", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))            
                norm_ROCy.append(max(valorsy)-min(valorsy))
                norm_ROCx.append(max(valorsx)-min(valorsx))
            else:
                if 'ROC' in '\t'.join(df_dades_postu_FxFy.columns.tolist()[:-1]):                
                    valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROC", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))                            
                    valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("ROC", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))            
                    not_norm_ROCy.append(max(valorsy)-min(valorsy))
                    not_norm_ROCx.append(max(valorsx)-min(valorsx))     
                    
            if isinstance(df_dades[[27]].values[0][0], float) and df_dades[[27]].values[0][0] > 90:
                valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGA", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGA", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                norm_RGAy.append(max(valorsy)-min(valorsy))
                norm_RGAx.append(max(valorsx)-min(valorsx))
            else:
                if 'RGA' in '\t'.join(df_dades_postu_FxFy.columns.tolist()[:-1]):
                    valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGA", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGA", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                    not_norm_RGAy.append(max(valorsy)-min(valorsy))
                    not_norm_RGAx.append(max(valorsx)-min(valorsx))      
                    
            if isinstance(df_dades[[32]].values[0][0], float) and df_dades[[32]].values[0][0] > 80:
                valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGC", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGC", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                norm_RGCy.append(max(valorsy)-min(valorsy))
                norm_RGCx.append(max(valorsx)-min(valorsx))
            else:
                if 'RGC' in '\t'.join(df_dades_postu_FxFy.columns.tolist()[:-1]):
                    valorsy = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGC", df_dades_postu_FxFy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_FxFy[find_last_string_with_text("RGC", df_dades_postu_FxFy.columns.tolist()[:-1])[0:-1] + "x"].values.tolist()[0:1200]))
                    not_norm_RGCy.append(max(valorsy)-min(valorsy))
                    not_norm_RGCx.append(max(valorsx)-min(valorsx))                


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_ROAx, norm_ROAy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_ROAx, not_norm_ROAy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima força a la prova ROA")
ax1.set_xlabel("Força ML")
ax1.set_ylabel("Força AP")
plt.legend()


plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_ROCx, norm_ROCy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_ROCx, not_norm_ROCy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima força a la prova ROC")
ax1.set_xlabel("Força ML")
ax1.set_ylabel("Força AP")
plt.legend()

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_RGAx, norm_RGAy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_RGAx, not_norm_RGAy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxima força a la prova RGA")
ax1.set_xlabel("Força ML")
ax1.set_ylabel("Força AP")
plt.legend()

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_RGCx, norm_RGCy, s = 1, label="Pacients amb valoració >= 80")
ax1.scatter(not_norm_RGCx, not_norm_RGCy, s = 1, label="Pacients amb valoració < 80")
ax1.set_title("Màxima força a la prova RGC")
ax1.set_xlabel("Força ML")
ax1.set_ylabel("Força AP")
plt.legend()

plt.show()

In [ ]:
norm_ROA = np.vstack((norm_ROAx, norm_ROAy)).T
norm_ROA = norm_ROA.tolist()

not_norm_ROA = np.vstack((not_norm_ROAx, not_norm_ROAy)).T
not_norm_ROA = not_norm_ROA.tolist()

dist_norm = []
dist_not_norm = []

for p in norm_ROA:
    dist_norm.append(distancia_al_limite_de_la_normalidad(p, norm_ROA))

for p in not_norm_ROA:
    dist_not_norm.append(distancia_al_limite_de_la_normalidad(p, norm_ROA))    

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(range(0,len(dist_norm)), dist_norm, s = 1, label="Dades normals")
ax1.scatter(range(0,len(dist_not_norm)), dist_not_norm, s = 1, label="Dades no normals")
ax1.set_title("Comparació distància al límit de la normalitat ROA")
plt.legend()

plt.show()

In [ ]:
norm_ROC = np.vstack((norm_ROCx, norm_ROCy)).T
norm_ROC = norm_ROC.tolist()

not_norm_ROC = np.vstack((not_norm_ROCx, not_norm_ROCy)).T
not_norm_ROC = not_norm_ROC.tolist()

dist_norm = []
dist_not_norm = []

for p in norm_ROC:
    dist_norm.append(distancia_al_limite_de_la_normalidad(p, norm_ROC))

for p in not_norm_ROC:
    dist_not_norm.append(distancia_al_limite_de_la_normalidad(p, norm_ROC))    

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(range(0,len(dist_norm)), dist_norm, s = 1, label="Dades normals")
ax1.scatter(range(0,len(dist_not_norm)), dist_not_norm, s = 1, label="Dades no normals")
ax1.set_title("Comparació distància al límit de la normalitat ROC")
plt.legend()

plt.show()

In [ ]:
norm_RGA = np.vstack((norm_RGAx, norm_RGAy)).T
norm_RGA = norm_RGA.tolist()

not_norm_RGA = np.vstack((not_norm_RGAx, not_norm_RGAy)).T
not_norm_RGA = not_norm_RGA.tolist()

dist_norm = []
dist_not_norm = []

for p in norm_RGA:
    dist_norm.append(distancia_al_limite_de_la_normalidad(p, norm_RGA))

for p in not_norm_RGA:
    dist_not_norm.append(distancia_al_limite_de_la_normalidad(p, norm_RGA))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(range(0,len(dist_norm)), dist_norm, s = 1, label="Dades normals")
ax1.scatter(range(0,len(dist_not_norm)), dist_not_norm, s = 1, label="Dades no normals")
ax1.set_title("Comparació distància al límit de la normalitat RGA")
plt.legend()

plt.show()

In [ ]:
norm_RGC = np.vstack((norm_RGCx, norm_RGCy)).T
norm_RGC = norm_RGC.tolist()

not_norm_RGC = np.vstack((not_norm_RGCx, not_norm_RGCy)).T
not_norm_RGC = not_norm_RGC.tolist()

dist_norm = []
dist_not_norm = []

for p in norm_RGC:
    dist_norm.append(distancia_al_limite_de_la_normalidad(p, norm_RGC))

for p in not_norm_RGC:
    dist_not_norm.append(distancia_al_limite_de_la_normalidad(p, norm_RGC))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(range(0,len(dist_norm)), dist_norm, s = 1, label="Dades normals")
ax1.scatter(range(0,len(dist_not_norm)), dist_not_norm, s = 1, label="Dades no normals")
ax1.set_title("Comparació distància al límit de la normalitat RGC")
plt.legend()

plt.show()

In [ ]:
import os

norm_ROAx = []
norm_ROAy = []

norm_ROCx = []
norm_ROCy = []

norm_RGAx = []
norm_RGAy = []

norm_RGCx = []
norm_RGCy = []

not_norm_ROAx = []
not_norm_ROAy = []

not_norm_ROCx = []
not_norm_ROCy = []

not_norm_RGAx = []
not_norm_RGAy = []

not_norm_RGCx = []
not_norm_RGCy = []

edats = []

for filename in os.listdir("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf"):
    if filename.endswith('.txt'):
        reader = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf/" + filename, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None)
        df_dades = reader.get_chunk()
        if df_dades[[9]].values[0][0] > 67:
            edats.append(df_dades[[9]].values[0][0])
            num_test = df_dades[[16]].values[0] + df_dades[[21]].values[0] + df_dades[[26]].values[0] + df_dades[[31]].values[0]
            df_dades_postu_xy = pd.read_csv("C:/Users/Usuario/OneDrive/Escritorio/Universitat/TFG/Mataro_posturograf/" + filename, delimiter="\t", encoding="ansi",skiprows=(17+int(num_test)), chunksize=1200, index_col=False).get_chunk()
            
            #if isinstance(df_dades[[18]].values[0][0], float) and df_dades[[18]].values[0][0] > 90:
            if isinstance(df_dades[[17]].values[0][0], float) and df_dades[[17]].values[0][0] > 90:     
                valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROA", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROA", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                norm_ROAy.append(max(valorsy)-min(valorsy))
                norm_ROAx.append(max(valorsx)-min(valorsx))
            else:
                if 'ROA' in '\t'.join(df_dades_postu_xy.columns.tolist()[:-1]):                
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROA", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROA", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                    not_norm_ROAy.append(max(valorsy)-min(valorsy))
                    not_norm_ROAx.append(max(valorsx)-min(valorsx))
                        
            #if isinstance(df_dades[[23]].values[0][0], float) and df_dades[[23]].values[0][0] > 90:                    
            if isinstance(df_dades[[22]].values[0][0], float) and df_dades[[22]].values[0][0] > 90:
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROC", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))                            
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROC", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))            
                    norm_ROCy.append(max(valorsy)-min(valorsy))
                    norm_ROCx.append(max(valorsx)-min(valorsx))
            else:
                if 'ROC' in '\t'.join(df_dades_postu_xy.columns.tolist()[:-1]):                
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROC", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))                            
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("ROC", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))            
                    not_norm_ROCy.append(max(valorsy)-min(valorsy))
                    not_norm_ROCx.append(max(valorsx)-min(valorsx)) 
                        
            #if isinstance(df_dades[[28]].values[0][0], float) and df_dades[[28]].values[0][0] > 90:
            if isinstance(df_dades[[27]].values[0][0], float) and df_dades[[27]].values[0][0] > 90:

                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGA", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGA", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                    norm_RGAy.append(max(valorsy)-min(valorsy))
                    norm_RGAx.append(max(valorsx)-min(valorsx))
            else:
                if 'RGA' in '\t'.join(df_dades_postu_xy.columns.tolist()[:-1]):
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGA", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGA", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                    not_norm_RGAy.append(max(valorsy)-min(valorsy))
                    not_norm_RGAx.append(max(valorsx)-min(valorsx))   
                        
            #if isinstance(df_dades[[33]].values[0][0], float) and df_dades[[33]].values[0][0] > 90:                    
            if isinstance(df_dades[[32]].values[0][0], float) and df_dades[[32]].values[0][0] > 90:
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGC", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGC", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                    norm_RGCy.append(max(valorsy)-min(valorsy))
                    norm_RGCx.append(max(valorsx)-min(valorsx))
            else:
                if 'RGC' in '\t'.join(df_dades_postu_xy.columns.tolist()[:-1]):
                    valorsy = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGC", df_dades_postu_xy.columns.tolist()[:-1])].values.tolist()[0:1200]))
                    valorsx = list(map(float, df_dades_postu_xy[find_last_string_with_text("RGC", df_dades_postu_xy.columns.tolist()[:-1])[0:-1] + "X"].values.tolist()[0:1200]))
                    not_norm_RGCy.append(max(valorsy)-min(valorsy))
                    not_norm_RGCx.append(max(valorsx)-min(valorsx))                


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_ROAx, norm_ROAy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_ROAx, not_norm_ROAy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova ROA")
plt.legend()

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_ROCx, norm_ROCy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_ROCx, not_norm_ROCy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova ROC")
plt.legend()

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_RGAx, norm_RGAy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_RGAx, not_norm_RGAy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova RGA")
plt.legend()

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(norm_RGCx, norm_RGCy, s = 1, label="Pacients amb valoració >= 90")
ax1.scatter(not_norm_RGCx, not_norm_RGCy, s = 1, label="Pacients amb valoració < 90")
ax1.set_title("Màxim desplaçament a la prova RGC")
plt.legend()

plt.show()